# DataJoint U24 - Workflow DeepLabCut

## Workflow Automation

In the previous notebook [03-Process](./03-Process.ipynb), we ran through the workflow in detailed steps. For daily running routines, the current notebook provides a more succinct and automatic approach to run through the pipeline using some utility functions in the workflow.

The commands here run a workflow using [example data](https://downgit.github.io/#/home?url=https://github.com/DeepLabCut/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30) from the [00-DownloadData](./00-DataDownload_Optional.ipynb) notebook, but note where placeholders could be changed for a different dataset.

In [10]:
import os; from pathlib import Path
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='workflow-deeplabcut', ("Please move to the "
                                                              + "workflow directory")

## Ingestion of subjects, sessions, videos and training parameters

Refer to the `user_data` folder in the workflow.

1. Fill subject and session information in files `subjects.csv` and `sessions.csv`
2. Fill in recording and parameter information in `recordings.csv` and `config_params.csv`
    + Add both training and estimation videos to the recording list
    + Additional columns in `config_params.csv` will be treated as model training parameters
3. Run automatic scripts prepared in `workflow_deeplabcut.ingest` for ingestion: 
    + `ingest_subjects` for `subject.Subject`
    + `ingest_sessions` - for session tables `Session`, `SessionDirectory`, and `SessionNote`
    + `ingest_dlc_items` - for DLC tables `VideoRecording` and `ModelTrainingParamSet`

In [11]:
from workflow_deeplabcut.pipeline import lab, subject, session, dlc
from workflow_deeplabcut.ingest import ingest_subjects, ingest_sessions, ingest_dlc_items
ingest_subjects(); ingest_sessions(); ingest_dlc_items()


---- Inserting 0 entry(s) into subject ----

---- Inserting 0 entry(s) into session ----

---- Inserting 0 entry(s) into session_directory ----

---- Inserting 0 entry(s) into session_note ----

---- Inserting 0 entry(s) into #model_training_param_set ----

---- Inserting 0 entry(s) into video_recording ----

---- Inserting 0 entry(s) into video_recording__file ----


## Setting project variables

1. Set your root directory in your DataJoint config file, under `custom` as `dlc_root_data_dir`

In [12]:
import datajoint as dj; dj.config.load('dj_local_conf.json')
from element_interface.utils import find_full_path
data_dir = find_full_path(dj.config['custom']['dlc_root_data_dir'], # root from config
                          'openfield-Pranav-2018-10-30')            # DLC project dir
config_path = (data_dir / 'config.yaml')
from deeplabcut.create_project.demo_data import load_demo_data as dlc_load_demo
dlc_load_demo(config_path)

AttributeError: 'CommentedSeq' object has no attribute 'keys'

In [ ]:
%debug

> /Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/deeplabcut/create_project/demo_data.py(62)transform_data()
     60         print("This is not an offical demo dataset.")
     61 
---> 62     if "WILL BE AUTOMATICALLY UPDATED BY DEMO CODE" in cfg["video_sets"].keys():
     63         cfg["video_sets"][str(video_file)] = cfg["video_sets"].pop(
     64             "WILL BE AUTOMATICALLY UPDATED BY DEMO CODE"



ipdb>  up


> /Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/deeplabcut/create_project/demo_data.py(39)load_demo_data()
     37     config = str(config)
     38 
---> 39     transform_data(config)
     40     if createtrainingset:
     41         print("Loaded, now creating training data...")



ipdb>  


> /var/folders/_9/tzvq__ws5z9gv5s7jvkj570r0000gn/T/ipykernel_8305/2210132270.py(7)<module>()
      3 data_dir = find_full_path(dj.config['custom']['dlc_root_data_dir'], # root from config
      4                           'openfield-Pranav-2018-10-30')            # DLC project dir
      5 config_path = (data_dir / 'config.yaml')
      6 from deeplabcut.create_project.demo_data import load_demo_data as dlc_load_demo
----> 7 dlc_load_demo(config_path)



...

ordereddict([('Task', 'openfield'), ('TrainingFraction', [0.95]), ('alphavalue', 0.7), ('batch_size', 4), ('bodyparts', ['snout', 'leftear', 'rightear', 'tailbase']), ('colormap', 'jet'), ('corner2move2', [50, 50]), ('cropping', False), ('date', 'Oct30'), ('default_augmenter', 'imgaug'), ('default_net_type', 'resnet_50'), ('dotsize', 8), ('filter_type', ''), ('identity', None), ('iteration', 0), ('maxiters', '5'), ('modelprefix', ''), ('move2corner', True), ('multianimalproject', None), ('numframes2pick', 20), ('pcutoff', 0.4), ('project_path', '/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30'), ('scorer', 'Pranav'), ('scorer_legacy', 'False'), ('shuffle', '1'), ('skeleton', []), ('skeleton_color', 'black'), ('snapshotindex', -1), ('start', 0), ('stop', 1), ('track_method', ''), ('train_float', 0.95), ('trainingsetindex', '0'), ('video_sets', ['/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/videos/m3v1

ipdb>  cfg["video_sets"].keys()


*** AttributeError: 'CommentedSeq' object has no attribute 'keys'


ipdb>  cfg["video_sets"]


['/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4']


2. For this demo, we generate a copy to show pose estimation. This is recording_id 2 in `recordings.csv`

In [9]:
vid_path = str(data_dir).replace(" ", "\ ") + '/videos/m3v1mp4'
cmd = (f'ffmpeg -n -hide_banner -loglevel error -ss 0 -t 2 -i {vid_path}.mp4 -vcodec copy '
       + f'-acodec copy {vid_path}-copy.mp4') # New video copy, first 2 seconds
os.system(cmd)

File '/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/videos/m3v1mp4-copy.mp4' already exists. Exiting.


256

3. Pair training video with training parameters, and launch training.

In [ ]:
key=(dlc.VideoRecording&'recording_id=1').fetch1('KEY') # replace w/relevant IDs 
key.update({'paramset_idx':1,'training_id':1,
            'project_path':'openfield-Pranav-2018-10-30/'})
dlc.TrainingTask.insert1(key, skip_duplicates=True)
dlc.TrainingTask.populate()

4. Add this model to the `Model` table and evaluate.

In [ ]:
dlc.Model.insert_new_model(model_name='OpenField-5',dlc_config=dlc_config_path,
                           shuffle=1,trainingsetindex=0,
                           model_description='Open field model trained 5 iterations',
                           body_part_descriptions = bp_desc,paramset_idx=1)
dlc.ModelEvaluation.populate()

5. Add a pose estimation task, and launch pose estimation.

In [ ]:
key=(dlc.VideoRecording&'recording_id=2').fetch1('KEY') # change relevant ID
key.update({'model_name': 'OpenField-1010', 'task_mode': 'trigger'})
analyze_params={'save_as_csv':True} # add any others from deeplabcut.analyze_videos
dlc.PoseEstimationTask.insert_estimation_task(key,params=analyze_params,
                                              skip_duplicates=True)
dlc.PoseEstimation.populate()

6. Retrieve estimated position data.

In [ ]:
dlc.PoseEstimation.get_trajectory(key)

## Summary and next step

+ This notebook runs through the workflow in an automatic manner.

+ In the next notebook [05-explore](05-explore.ipynb), we will introduce how to query, fetch and visualize the contents we ingested into the tables.